In [1]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()

endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
api_key = os.environ["AZURE_OPENAI_API_KEY"]

client = AzureOpenAI(
        azure_endpoint = endpoint, 
        api_key=api_key,
        api_version="2024-02-01"
    )

CHAT_COMPLETIONS_MODEL = "gpt-4o-mini"

# A Few Shot Learning

In [2]:
# Zero-shot classification
system_prompt ="""Predict up to 5 emojis as a response to a text chat message. The output
should only include emojis.

input: The new visual design is blowing my mind 🤯
output: ➕,💘, ❤‍🔥

input: Well that looks great regardless
output: ❤️,🪄

input: Unfortunately this won't work
output: 💔,😔

input: sounds good, I'll look into that
output: 🙏,👍

input: 10hr cut of jeff goldblum laughing URL
output: 😂,💀,⚰️
"""
user_prompt = "The new user interface is amazing!"
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":system_prompt},
                {"role":"user","content": user_prompt,}])
print(response.choices[0].message.content)

😍,👏,✨,🎉,💖


# Prompt Engineering Best Practices

## Write clear instructions

Examples:

-----------------------
Prompt:

Write code to calculate the Fibonacci sequence.

Better:

Write a TypeScript function to efficiently calculate the Fibonacci sequence. Comment the code liberally to explain what each piece does and why it's written that way.

----------------------

Prompt:

Summarize the meeting notes.

Better:

Summarize the meeting notes in a single paragraph. Then write a markdown list of the speakers and each of their key points. Finally, list the next steps or action items suggested by the speakers, if any.


## Role Playing

Examples:

-----------------------

System Message: When I ask for help to write something, you will reply with a document that contains at least one joke or playful comment in every paragraph.

----------------------


## Segment input text

Examples:

------------------------

user message: Summarize the text delimited by triple quotes with a haiku.

"""insert text here"""

------------------------

system message: You will be provided with a pair of articles (delimited with XML tags) about the same topic. First summarize the arguments of each article. Then indicate which of them makes a better argument and explain why.

user message: 

\<article> insert first article here \</article>

\<article> insert second article here \</article>

------------------------

## Explain steps and processes to complete a task

Examples:

--------------------------------

System Message:
Use the following step-by-step instructions to respond to user inputs.

Step 1 - The user will provide you with text in triple quotes. Summarize this text in one sentence with a prefix that says "Summary: ".

Step 2 - Translate the summary from Step 1 into Spanish, with a prefix that says "Translation: ".

---------------------------------

## Ejercicio 1: Clasificación de Emojis con Zero-shot
Enunciado:
Dado el siguiente texto, predice hasta 5 emojis que mejor representen la emoción o el tema del mensaje. Usa el modelo de Zero-shot para hacer la clasificación.

Texto: "Estoy tan feliz que no puedo dejar de sonreír."

Resultado esperado: 😄,😊,✨,🌞,❤️

In [8]:
user_prompt = "Estoy tan feliz que no puedo dejar de sonreír."

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are an emoji classification assistant."},
        {"role": "user", "content": f"Texto: {user_prompt}. ¿Cuáles son los 5 emojis que mejor representan la emoción o el tema del mensaje?, muestra solo los emojis"},
    ],
    max_tokens=70,
)

print(response.choices[0].message.content)


😄😊😀😁🥳


## Ejercicio 2: Traducción de Resumen
Enunciado:
El siguiente texto es una transcripción de una reunión. Resúmelo en una sola oración, luego traduce ese resumen al inglés.

Texto:
"""Hoy discutimos sobre la necesidad de mejorar la interfaz de usuario en el sitio web. Los participantes estuvieron de acuerdo en que se debe hacer más intuitiva y accesible, especialmente para usuarios mayores. También se mencionó la importancia de agregar soporte multilingüe."""



In [9]:
# Texto a resumir
user_prompt = """Hoy discutimos sobre la necesidad de mejorar la interfaz de usuario en el sitio web. Los participantes estuvieron de acuerdo en que se debe hacer más intuitiva y accesible, especialmente para usuarios mayores. También se mencionó la importancia de agregar soporte multilingüe."""

# Paso 1: Resumir el texto
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a summarization assistant."},
        {"role": "user", "content": f"Resume el siguiente texto en una sola oración: {user_prompt}"}
    ],
    max_tokens=100,
)
summary = response.choices[0].message.content
print(f"Summary: {summary}")

# Paso 2: Traducir el resumen al inglés
translation_prompt = f"Traduce el siguiente resumen al inglés: {summary}"
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a translation assistant."},
        {"role": "user", "content": translation_prompt}
    ],
    max_tokens=100,
)
print(f"Translation: {response.choices[0].message.content}")



Summary: Hoy se conversó sobre la importancia de mejorar la interfaz de usuario del sitio web para hacerla más intuitiva y accesible, especialmente para usuarios mayores, y se destacó la necesidad de incorporar soporte multilingüe.
Translation: Today, the discussion focused on the importance of enhancing the website's user interface to make it more intuitive and accessible, especially for older users, and the need to incorporate multilingual support was emphasized.


## Ejercicio 4: Respuesta Automática con Role-playing
Enunciado:
Configura un modelo para responder a una solicitud de escritura, haciendo que la respuesta contenga un toque de humor. Usa la siguiente solicitud como ejemplo.

Solicitud:
"Escribe una carta formal solicitando días de vacaciones."



In [10]:
# Instrucciones para el modelo (system_prompt) para darle el toque de humor a la carta
system_prompt = """
Eres un asistente de escritura profesional. Ayuda al usuario a escribir una carta formal solicitando días de vacaciones. La carta debe ser respetuosa y clara, pero también debe incluir un toque de humor sutil para hacerla más ligera y agradable. 
"""

user_prompt = "Escribe una carta formal solicitando días de vacaciones."

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages=[{"role": "system", "content": system_prompt},
              {"role": "user", "content": user_prompt}]
)

# Mostrar la respuesta generada
print(response.choices[0].message.content)  # Ejemplo de salida: carta con broma



[Tu Nombre]  
[Tu Dirección]  
[Ciudad, Estado, Código Postal]  
[Tu Correo Electrónico]  
[Tu Número de Teléfono]  
[Fecha]  

[Nombre del Jefe o Superior]  
[Nombre de la Empresa]  
[Dirección de la Empresa]  
[Ciudad, Estado, Código Postal]  

Estimado/a [Nombre del Jefe o Superior]:

Espero que este mensaje le encuentre disfrutando de un café bien cargado y en plena forma para enfrentar el día. 

Me dirijo a usted para solicitar formalmente algunos días de vacaciones, ya que creo que mi cerebro y mi cuerpo han llegado a un acuerdo mutuo: es necesario tomar un descanso antes de que comiencen a planear su jubilación. Mis días de vacaciones solicitados son del [fecha de inicio] al [fecha de finalización], y me aseguraré de que todas mis responsabilidades estén al día antes de mi partida. Esto incluye dejar todo en perfecto orden, como un maestro de ceremonias que se asegura de que todo esté preparado para el gran evento.

Estoy seguro/a de que el equipo podrá manejar el flujo de traba

## Ejercicio 5: Segmentación de Texto
Enunciado:
Segmenta un texto delimitado por comillas triples y luego tradúcelo a otro idioma (por ejemplo, inglés). Aplica el siguiente formato de entrada de texto y muestra cómo lo harías.

Texto de Entrada:
"""Hoy es un buen día para aprender nuevas cosas y mejorar nuestras habilidades. ¡El futuro está lleno de oportunidades!"""


In [11]:
# Instrucciones para segmentar el texto y luego traducirlo
system_prompt = """
Eres un asistente de traducción. Tu tarea es dividir un texto largo en segmentos más pequeños y luego traducir cada segmento al inglés. Asegúrate de que cada fragmento mantenga el contexto y el significado original. Después de segmentar, traduce al inglés el texto segmentado.
"""

user_prompt = """Hoy es un buen día para aprender nuevas cosas y mejorar nuestras habilidades. ¡El futuro está lleno de oportunidades!"""

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages=[{"role": "system", "content": system_prompt},
              {"role": "user", "content": user_prompt}]
)

# Mostrar la respuesta generada
print(response.choices[0].message.content)  # Salida esperada: segmentación en español e inglés



Segmentación:

1. Hoy es un buen día para aprender nuevas cosas 
2. y mejorar nuestras habilidades.
3. ¡El futuro está lleno de oportunidades!

Traducción al inglés:

1. Today is a good day to learn new things 
2. and improve our skills.
3. The future is full of opportunities!


## Ejercicio 6: Predicción de Emojis con Zero-shot
Enunciado:
Realiza una clasificación Zero-shot utilizando el modelo y predice hasta 5 emojis para el siguiente mensaje:

Texto de entrada:
"Estoy muy cansado, pero contento con el trabajo que he hecho."



In [12]:
# Instrucciones para predecir emojis basados en el texto
system_prompt = """Predice hasta 5 emojis como respuesta a un mensaje de chat. La salida debe solo incluir emojis."""
user_prompt = "Estoy muy cansado, pero contento con el trabajo que he hecho."

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages=[{"role": "system", "content": system_prompt},
              {"role": "user", "content": user_prompt}]
)

# Mostrar la respuesta generada
print(response.choices[0].message.content)  # Ejemplo de salida: "😅,💪,👌"

😌💪🌟😴😊


## Ejercicio 7: Predicción de Emojis con Few-shot Learning

Enunciado:

En este ejercicio utilizarás Few-shot learning con roles (system, user, assistant) para predecir emojis. En Few-shot learning, proporcionamos ejemplos específicos de interacciones para que el modelo aprenda a generalizar la tarea con pocos ejemplos.

Dado un conjunto de ejemplos de interacciones entre un usuario y un asistente, utiliza el modelo para predecir hasta 5 emojis para el siguiente mensaje:

Mensaje: "Estoy muy cansado, pero contento con el trabajo que he hecho."

Devuelve solo los emojis, separados por comas.

In [13]:
# Definir el prompt con Few-shot utilizando múltiples roles
few_shot_prompt = [
    {"role": "system", "content": "You are an assistant that predicts emojis based on user text."},
    
    {"role": "user", "content": "Estoy muy feliz con mi nuevo trabajo."},
    {"role": "assistant", "content": "😊,🎉,💼,💪"},
    
    {"role": "user", "content": "No me siento bien, tengo dolor de cabeza."},
    {"role": "assistant", "content": "😞,🤕,💔"},
    
    {"role": "user", "content": "Hoy es un día increíble para correr al aire libre."},
    {"role": "assistant", "content": "🏃‍♂️,🌳,🌞,💨"},
    
    {"role": "user", "content": "Acabo de terminar de ver una película emocionante."},
    {"role": "assistant", "content": "🎬,👏,😱,❤️"},
    
    {"role": "user", "content": "Voy a dormir temprano, me siento agotado."},
    {"role": "assistant", "content": "😴,🛏️,😴"},
    
    # Ahora pedimos la predicción para el nuevo mensaje
    {"role": "user", "content": "Estoy muy cansado, pero contento con el trabajo que he hecho."},
]

# Llamada a la API para obtener la predicción de emojis
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages=few_shot_prompt
)

# Mostrar la respuesta generada
print(response.choices[0].message.content)  # Ejemplo de salida: "😅,💪,👌"

😌,💪,😊,💼
